scikit-hts showcase: M5 kaggle competition

In this notebook we will use the data from the Kaggle competiton to perform some hierarchical forecasting. The problem is particularly well suited for the library, as there is a clear hierarchical relationship between each of the series: we have states, stores, categories, departments, and items; sum of sales of items resolve to departments, which summed resolved to categories and so forth.

We will however limit the scope of the forecasting task to producing forecasts at the department level, rather than going to the full extent and forecasting for single items.

The reasons for this is that this notebook is designed for exemplification purposes, rather than providing a workable solution for the challenge.

reference: https://github.com/carlomazzaferro/scikit-hts-examples/blob/master/notebooks/M5.ipynb

Kaggle discussion on hts with Python: https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135449

In [1]:
import os
from pprint import pprint
import importlib

import pandas
import numpy
import matplotlib
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from pandas.core.common import SettingWithCopyWarning

In [2]:
import json

In [3]:
from hts.hierarchy import HierarchyTree

/project/project/venv/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Importing plotly failed. Interactive plots will not work.


In [36]:
from hts import HTSRegressor

In [6]:
!pwd

/project/project/notebooks


In [5]:
!ls

'20200508(1).ipynb'	      sales_train_validation.csv.zip
'20200508(2).ipynb'	      sample_submission.csv
'20200517(1).ipynb'	      sample_submission.csv.zip
'20200518(1).ipynb'	      sell_prices.csv
'20200518(2).ipynb'	      sell_prices.csv.zip
 Untitled1.ipynb	      skhts.ipynb
 calendar.csv		      submission.csv
 docker_command.txt	      test_from_1858_till_1913.ipynb
 model.txt		      test_from_1914_till_1969.ipynb
 processed_data		      train.bin
 sales.csv		      url.txt
 sales_train_validation.csv   valid_data_label.npz


In [7]:
# Download the raw files into this directory
data = './'

In [8]:
train = pandas.read_csv(os.path.join(data, 'sales_train_validation.csv'), 
                        encoding='utf-8', 
                        engine='c')# .drop('item_id', 1)

# Ensures uniqueness of category, dept, and item across hierarchie
train['cat_id']  = (train['store_id'] + '_' + train['cat_id'])
train['dept_id'] = (train['store_id'] + '_' + train['dept_id'])
train['id'] = (train['store_id'] + '_' + train['id'])

In [9]:
train

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,CA_1_HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,CA_1_HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,CA_1_HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,CA_1_HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,CA_1_HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,WI_3_FOODS_3_823_WI_3_validation,FOODS_3_823,WI_3_FOODS_3,WI_3_FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,WI_3_FOODS_3_824_WI_3_validation,FOODS_3_824,WI_3_FOODS_3,WI_3_FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,WI_3_FOODS_3_825_WI_3_validation,FOODS_3_825,WI_3_FOODS_3,WI_3_FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,WI_3_FOODS_3_826_WI_3_validation,FOODS_3_826,WI_3_FOODS_3,WI_3_FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [10]:
calendar = pandas.read_csv(os.path.join(data, 'calendar.csv'))

day_cols = [col for col in train.columns if col.startswith('d_')]
idx = [int(col.split('d_')[1]) for col in day_cols]

# will be useful later
train_date_id = pandas.to_datetime(calendar[calendar.d.apply(lambda x: int(x.split('d_')[1])).isin(idx)].date)

In [11]:
train_date_id

0      2011-01-29
1      2011-01-30
2      2011-01-31
3      2011-02-01
4      2011-02-02
          ...    
1908   2016-04-20
1909   2016-04-21
1910   2016-04-22
1911   2016-04-23
1912   2016-04-24
Name: date, Length: 1913, dtype: datetime64[ns]

In [12]:
def transpose(column, index, day_col):
    """
    Turn the row oriented time series into a column oriented one 
    """
    ts = []
    new_cols = train[column].unique()
    
    for value in new_cols:
        value_ts = train[train[column] == value]
        vertical = value_ts[day_col].sum().T
        vertical.index = index
        ts.append(vertical)
    return pandas.DataFrame({k: v for k, v in zip(new_cols, ts)})

In [13]:
state_ts = transpose('state_id', train_date_id, day_cols)
store_ts = transpose('store_id', train_date_id, day_cols)
cat_ts = transpose('cat_id', train_date_id, day_cols)
dept_ts = transpose('dept_id', train_date_id, day_cols)

In [14]:
state_ts

,CA,TX,WI
date,,,
2011-01-29,14195,9438,8998
2011-01-30,13805,9630,8314
2011-01-31,10108,6778,6897
2011-02-01,11047,7381,6984
2011-02-02,9925,5912,3309
...,...,...,...
2016-04-20,15148,9600,10595
2016-04-21,14488,9602,10943
2016-04-22,17095,10615,12807


In [15]:
dept_ts

,CA_1_HOBBIES_1,CA_1_HOBBIES_2,CA_1_HOUSEHOLD_1,CA_1_HOUSEHOLD_2,CA_1_FOODS_1,CA_1_FOODS_2,CA_1_FOODS_3,CA_2_HOBBIES_1,CA_2_HOBBIES_2,CA_2_HOUSEHOLD_1,...,WI_2_FOODS_1,WI_2_FOODS_2,WI_2_FOODS_3,WI_3_HOBBIES_1,WI_3_HOBBIES_2,WI_3_HOUSEHOLD_1,WI_3_HOUSEHOLD_2,WI_3_FOODS_1,WI_3_FOODS_2,WI_3_FOODS_3
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,528,28,361,181,297,674,2268,522,16,529,...,191,155,1269,256,22,584,148,152,583,2293
2011-01-30,489,9,350,170,284,655,2198,381,16,461,...,184,128,1121,342,14,541,195,138,585,2383
2011-01-31,409,6,279,114,214,396,1398,352,16,306,...,154,199,1233,228,20,420,106,127,575,1841
2011-02-01,383,9,278,123,175,476,1607,344,6,270,...,205,244,1564,183,11,327,94,98,533,1965
2011-02-02,263,5,195,135,182,354,1496,283,13,217,...,96,147,724,70,4,151,53,87,340,1427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-20,333,30,665,204,287,423,1780,303,41,586,...,335,830,1840,202,42,535,148,180,473,1579
2016-04-21,380,30,580,186,297,365,1871,232,39,521,...,307,851,1920,182,21,573,142,227,411,1670
2016-04-22,471,56,695,205,331,489,2140,425,60,778,...,400,953,2066,264,32,701,166,261,507,1897


In [16]:
item_ts = transpose('id', train_date_id, day_cols)

In [17]:
item_ts

,CA_1_HOBBIES_1_001_CA_1_validation,CA_1_HOBBIES_1_002_CA_1_validation,CA_1_HOBBIES_1_003_CA_1_validation,CA_1_HOBBIES_1_004_CA_1_validation,CA_1_HOBBIES_1_005_CA_1_validation,CA_1_HOBBIES_1_006_CA_1_validation,CA_1_HOBBIES_1_007_CA_1_validation,CA_1_HOBBIES_1_008_CA_1_validation,CA_1_HOBBIES_1_009_CA_1_validation,CA_1_HOBBIES_1_010_CA_1_validation,...,WI_3_FOODS_3_818_WI_3_validation,WI_3_FOODS_3_819_WI_3_validation,WI_3_FOODS_3_820_WI_3_validation,WI_3_FOODS_3_821_WI_3_validation,WI_3_FOODS_3_822_WI_3_validation,WI_3_FOODS_3_823_WI_3_validation,WI_3_FOODS_3_824_WI_3_validation,WI_3_FOODS_3_825_WI_3_validation,WI_3_FOODS_3_826_WI_3_validation,WI_3_FOODS_3_827_WI_3_validation
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
2011-01-30,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0
2011-01-31,0,0,0,0,0,0,0,0,7,1,...,0,5,1,0,2,2,0,0,0,0
2011-02-01,0,0,0,0,0,0,0,0,3,0,...,0,6,1,0,5,2,0,2,0,0
2011-02-02,0,0,0,0,0,0,0,0,0,0,...,0,5,1,0,2,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-20,1,1,1,0,1,0,1,4,0,0,...,4,1,1,0,0,0,0,1,1,0
2016-04-21,3,0,0,1,2,0,0,6,0,0,...,2,3,3,0,2,1,0,0,0,0
2016-04-22,0,0,1,3,2,2,0,3,0,2,...,0,1,6,0,3,0,0,0,3,0


In [18]:
df = pandas.concat([state_ts, store_ts, cat_ts,dept_ts, item_ts], 1)

# Total column is the root node -- the sum of of all demand across all stores (we have data on, at least)
df['total'] = df['CA'] + df['TX'] + df['WI']

In [19]:
df

,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,...,WI_3_FOODS_3_819_WI_3_validation,WI_3_FOODS_3_820_WI_3_validation,WI_3_FOODS_3_821_WI_3_validation,WI_3_FOODS_3_822_WI_3_validation,WI_3_FOODS_3_823_WI_3_validation,WI_3_FOODS_3_824_WI_3_validation,WI_3_FOODS_3_825_WI_3_validation,WI_3_FOODS_3_826_WI_3_validation,WI_3_FOODS_3_827_WI_3_validation,total
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,14195,9438,8998,4337,3494,4739,1625,2556,3852,3030,...,14,1,0,4,0,0,0,0,0,32631
2011-01-30,13805,9630,8314,4155,3046,4827,1777,2687,3937,3006,...,11,1,0,4,0,0,6,0,0,31749
2011-01-31,10108,6778,6897,2816,2121,3785,1386,1822,2731,2225,...,5,1,0,2,2,0,0,0,0,23783
2011-02-01,11047,7381,6984,3051,2324,4232,1440,2258,2954,2169,...,6,1,0,5,2,0,2,0,0,25412
2011-02-02,9925,5912,3309,2630,1942,3817,1536,1694,2492,1726,...,5,1,0,2,0,0,2,0,0,19146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-20,15148,9600,10595,3722,3691,5235,2500,2901,3384,3315,...,1,1,0,0,0,0,1,1,0,35343
2016-04-21,14488,9602,10943,3709,3303,5018,2458,2776,3446,3380,...,3,3,0,2,1,0,0,0,0,35033
2016-04-22,17095,10615,12807,4387,4457,5623,2628,3022,3902,3691,...,1,6,0,3,0,0,0,3,0,40517


In [17]:
#df.to_csv('./processed_data/M5_hierarchy.csv')

In [20]:
states = train.state_id.unique()
stores = train.store_id.unique()
depts = train.dept_id.unique()
cats = train.cat_id.unique()
items = train.id.unique()

# Here we build the tree as a dictionary. Each node (key in dict) has a list of 
# children value in dict, which in turn may also be a key in the dict, and have 
# children as well 
total = {'total': list(states)}
state_h = {k: [v for v in stores if v.startswith(k)] for k in states}
store_h = {k: [v for v in cats if v.startswith(k)] for k in stores}
dept_h = {k: [v for v in depts if v.startswith(k)] for k in cats}
item_h = {k: [v for v in items if v.startswith(k)] for k in depts}

In [21]:
total

{'total': ['CA', 'TX', 'WI']}

In [22]:
state_h

{'CA': ['CA_1', 'CA_2', 'CA_3', 'CA_4'],
 'TX': ['TX_1', 'TX_2', 'TX_3'],
 'WI': ['WI_1', 'WI_2', 'WI_3']}

In [23]:
store_h

{'CA_1': ['CA_1_HOBBIES', 'CA_1_HOUSEHOLD', 'CA_1_FOODS'],
 'CA_2': ['CA_2_HOBBIES', 'CA_2_HOUSEHOLD', 'CA_2_FOODS'],
 'CA_3': ['CA_3_HOBBIES', 'CA_3_HOUSEHOLD', 'CA_3_FOODS'],
 'CA_4': ['CA_4_HOBBIES', 'CA_4_HOUSEHOLD', 'CA_4_FOODS'],
 'TX_1': ['TX_1_HOBBIES', 'TX_1_HOUSEHOLD', 'TX_1_FOODS'],
 'TX_2': ['TX_2_HOBBIES', 'TX_2_HOUSEHOLD', 'TX_2_FOODS'],
 'TX_3': ['TX_3_HOBBIES', 'TX_3_HOUSEHOLD', 'TX_3_FOODS'],
 'WI_1': ['WI_1_HOBBIES', 'WI_1_HOUSEHOLD', 'WI_1_FOODS'],
 'WI_2': ['WI_2_HOBBIES', 'WI_2_HOUSEHOLD', 'WI_2_FOODS'],
 'WI_3': ['WI_3_HOBBIES', 'WI_3_HOUSEHOLD', 'WI_3_FOODS']}

In [24]:
dept_h

{'CA_1_HOBBIES': ['CA_1_HOBBIES_1', 'CA_1_HOBBIES_2'],
 'CA_1_HOUSEHOLD': ['CA_1_HOUSEHOLD_1', 'CA_1_HOUSEHOLD_2'],
 'CA_1_FOODS': ['CA_1_FOODS_1', 'CA_1_FOODS_2', 'CA_1_FOODS_3'],
 'CA_2_HOBBIES': ['CA_2_HOBBIES_1', 'CA_2_HOBBIES_2'],
 'CA_2_HOUSEHOLD': ['CA_2_HOUSEHOLD_1', 'CA_2_HOUSEHOLD_2'],
 'CA_2_FOODS': ['CA_2_FOODS_1', 'CA_2_FOODS_2', 'CA_2_FOODS_3'],
 'CA_3_HOBBIES': ['CA_3_HOBBIES_1', 'CA_3_HOBBIES_2'],
 'CA_3_HOUSEHOLD': ['CA_3_HOUSEHOLD_1', 'CA_3_HOUSEHOLD_2'],
 'CA_3_FOODS': ['CA_3_FOODS_1', 'CA_3_FOODS_2', 'CA_3_FOODS_3'],
 'CA_4_HOBBIES': ['CA_4_HOBBIES_1', 'CA_4_HOBBIES_2'],
 'CA_4_HOUSEHOLD': ['CA_4_HOUSEHOLD_1', 'CA_4_HOUSEHOLD_2'],
 'CA_4_FOODS': ['CA_4_FOODS_1', 'CA_4_FOODS_2', 'CA_4_FOODS_3'],
 'TX_1_HOBBIES': ['TX_1_HOBBIES_1', 'TX_1_HOBBIES_2'],
 'TX_1_HOUSEHOLD': ['TX_1_HOUSEHOLD_1', 'TX_1_HOUSEHOLD_2'],
 'TX_1_FOODS': ['TX_1_FOODS_1', 'TX_1_FOODS_2', 'TX_1_FOODS_3'],
 'TX_2_HOBBIES': ['TX_2_HOBBIES_1', 'TX_2_HOBBIES_2'],
 'TX_2_HOUSEHOLD': ['TX_2_HOUSEHOLD_1', 

In [25]:
hierarchy = {**total, **state_h, **store_h, **dept_h, **item_h}

In [26]:
with open('./processed_data/hierarchy.json', 'w') as j:
    json.dump(hierarchy, j)

#### A small digression

For exemplification purposes, here's what the hierarchy tree looks like. You are unlikely to need to build it yourself, it is instead built by the library automatically

In [27]:
ht = HierarchyTree.from_nodes(nodes=hierarchy, df=df)

# print(f'Root node: key: {ht.key}, children: {[k.key for k in ht.children]}')

# for child in ht.children:
#     print(f'{child.key} node\'s children: {[c.key for c in child]} ')

In [30]:
print(ht.children[0].key)

CA


In [32]:
# observe the recursive structure
print(f'Node: {ht.children[0].children[2].children[2].key}, item: ')
ht.children[0].children[2].children[2].item

Node: CA_3_FOODS, item: 


date
2011-01-29    3446
2011-01-30    3535
2011-01-31    2701
2011-02-01    3064
2011-02-02    2761
              ... 
2016-04-20    3286
2016-04-21    3056
2016-04-22    3409
2016-04-23    4459
2016-04-24    4698
Name: CA_3_FOODS, Length: 1913, dtype: int64

#### On to the fun part

We will fit Facebook's prophet to each of the nodes defined in our hierarchy tree, and then reconcile them using the (simplest) optimal reconciliation approach, namely ordinary least squares.

In [38]:
htsregressor = HTSRegressor

In [40]:
clf = HTSRegressor(model='prophet', revision_method='OLS', n_jobs=12)

TypeError: __init__() missing 5 required positional arguments: 'df', 'forecasts', 'mse', 'sum_mat', and 'transformer'

In [ ]:
model = clf.fit(df, hierarchy)